## Extracting data from logfiles

In [18]:
from log_parser import parse_keyboard_log
from keyboard_heatmap import KeyboardHeatmap

# Segment length of each data sample
seg_length = 300

# List of parsed logfiles
keyboard = []
keyboard.append(parse_keyboard_log("./logs/JON/SPY/keyboard_actions.log"))
keyboard.append(parse_keyboard_log("./logs/MAR/SOL/keyboard_actions.log"))
keyboard.append(parse_keyboard_log("./logs/HEN/SOL/keyboard_actions.log"))
keyboard.append(parse_keyboard_log("./logs/ZIR/SOL/keyboard_actions.log"))

# Empty lists for inserting data
X_actual = []
Y_actual = []

for k in range(len(keyboard)):
    for i in range(int(keyboard[k].time.iloc[-1] / seg_length)):
        # For each segment in each logfile
        # Create a heatmap for that segment
        heatmap = KeyboardHeatmap(keyboard[k], i, seg_length)
        # If the heatmap isn't blank
        if heatmap.class_label() != 'Null':
            X_actual.append(heatmap.heatmap_data().ravel().tolist())
            Y_actual.append(heatmap.class_label())

print(f"Data samples: {len(Y_actual)}")

Data samples: 48


## Training a ANN model on keyboard heatmap data

In [19]:
import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_actual, y_actual, random_state=0)

mlp = MLPClassifier(hidden_layer_sizes=(65,65), activation='logistic', solver='adam', max_iter=10000)
mlp.fit(X_train,y_train)

predict_train = mlp.predict(X_train)
predict_test = mlp.predict(X_test)


from sklearn.metrics import classification_report,confusion_matrix
print("Result on Training data")
print(confusion_matrix(y_train,predict_train))
print(classification_report(y_train,predict_train))
print("Result on Test data")
print(confusion_matrix(y_test,predict_test))
print(classification_report(y_test,predict_test))



Result on Training data
[[ 6  0  0  0]
 [ 0  8  0  0]
 [ 0  0 12  0]
 [ 0  0  0 10]]
              precision    recall  f1-score   support

      HENSOL       1.00      1.00      1.00         6
      JONSPY       1.00      1.00      1.00         8
      MARSOL       1.00      1.00      1.00        12
      ZIRSOL       1.00      1.00      1.00        10

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36

Result on Test data
[[6 0 0]
 [0 4 0]
 [0 0 2]]
              precision    recall  f1-score   support

      HENSOL       1.00      1.00      1.00         6
      JONSPY       1.00      1.00      1.00         4
      ZIRSOL       1.00      1.00      1.00         2

    accuracy                           1.00        12
   macro avg       1.00      1.00      1.00        12
weighted avg       1.00      1.00      1.00        12

